# FragNet baseline testing

Link: [Code](https://github.com/pnnl/FragNet) [Paper](https://arxiv.org/abs/2410.12156)

## Setup Instructions

In [1]:
!git clone https://github.com/pnnl/FragNet.git

Cloning into 'FragNet'...
remote: Enumerating objects: 322, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 322 (delta 152), reused 226 (delta 86), pack-reused 0 (from 0)
Receiving objects: 100% (322/322), 34.01 MiB | 1.53 MiB/s, done.
Resolving deltas: 100% (152/152), done.


In [2]:
!cd FragNet && pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.5/881.5 kB 5.1 MB/s  0:00:0036m-:--:--
  Preparing metadata (setup.py) ... done
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached charset_normalizer-3.4.3-cp310-cp310-macosx_10_9_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached tomli-2.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached MarkupSafe-3.0.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 3.9 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 

In [4]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.4.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cpu.html


In [7]:
!cd FragNet && pip install .

Processing /Users/htdvu/Desktop/myLab/myLab-HDAC6-results/notebooks/FragNet
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'fragnet' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fragnet'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for fragnet: filename=fragnet-0.1.0-py3-none-any.whl size=173215 sha256=c00bd337a1955b1118b0d310415b6766c63769f5bb3c12c83e4a0e805e31f334
  Stored in directory: /private/var/folders/n6/jfk0f88s4wd2kc9whb2j35dw0000gn/T/pip-ephem-wheel-cache-7ncraeba/wheels/b5/86/cb/907b999daa669cb84ac97576a6864045d14cd6b2d150c2a1a7
Successfully built fragnet


### Import necessary libraries

In [1]:
import os 

# enter the FragNet directory
os.chdir("Fragnet/fragnet")
print(os.getcwd())

/Users/htdvu/Desktop/myLab/myLab-HDAC6-results/notebooks/FragNet/fragnet


## Dataset
FragNet uses [ESOL](https://huggingface.co/datasets/scikit-fingerprints/MoleculeNet_ESOL) dataset from [Uni-Mol](https://github.com/deepmodeling/Uni-Mol/tree/main/unimol). 

ESOL is a dataset for predicting water solubility of molecules. The dataset contains 1128 molecules with their SMILES representations and corresponding solubility values (logS). The dataset is split into training, validation, and test sets with a ratio of 80:10:10.

In [2]:
!mkdir -p finetune_data/moleculenet/esol/raw/

### Installation and Data Preparation

In [4]:
!curl -o finetune_data/moleculenet/esol/raw/delaney-processed.csv https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/delaney-processed.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 96699  100 96699    0     0  83929      0  0:00:01  0:00:01 --:--:-- 83940


#### Create pretrain data

In [2]:
!python data_create/create_pretrain_datasets.py --save_path pretrain_data/esol --data_type exp1s --maxiters 500 --raw_data_path finetune_data/moleculenet/esol/raw/delaney-processed.csv

100%|███████████████████████████████████████| 1000/1000 [00:16<00:00, 59.67it/s]
16.771543741226196
0.004193782806396484
998it [00:12, 82.06it/s] 
(1000, 10)
100%|█████████████████████████████████████████| 128/128 [00:02<00:00, 63.08it/s]
2.029557943344116
0.0007376670837402344
128it [00:00, 328160.70it/s]
(128, 10)


#### Create finetune data

In [6]:
!python data_create/create_finetune_datasets.py --dataset_name moleculenet --dataset_subset esol --use_molebert True --output_dir finetune_data/moleculenet_exp1s --data_dir finetune_data/moleculenet --data_type exp1s

args.use_genes:  0
args.multi_conf_data:  0
Processing...
Done!
100%|█████████████████████████████████████████| 902/902 [00:11<00:00, 77.67it/s]
11.624279022216797
0.0052988529205322266
902it [00:04, 216.35it/s]
100%|█████████████████████████████████████████| 113/113 [00:02<00:00, 49.03it/s]
2.3047871589660645
0.000782012939453125
113it [00:00, 358514.64it/s]
100%|█████████████████████████████████████████| 113/113 [00:02<00:00, 42.47it/s]
2.661094903945923
0.0012328624725341797
113it [00:00, 4282.45it/s]


## Model

### Pretrain

In [7]:
!python train/pretrain/pretrain_gat2.py --config exps/pt/unimol_exp1s4/config.yaml

['pt_1000_1999.pkl', 'pt_0_999.pkl']
/Users/htdvu/miniconda3/envs/mylab-hdac6/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


## Finetune

In [8]:
!python train/finetune/finetune_gat2.py --config exps/ft/esol/e1pt4.yaml

gat2
using FTHead3
loaded from gat2
/Users/htdvu/Desktop/myLab/myLab-HDAC6-results/notebooks/FragNet/fragnet/train/finetune/finetune_gat2.py:225: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper